In [ ]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16, vgg19
import glob
import keras
from keras.layers import Flatten, Dense, Dropout

In [ ]:
dog_path = Path("training_data") / "dogs"
not_dog_path = Path("training_data") / "not_dogs"

In [ ]:
images = []
labels = []

In [ ]:
for img in dog_path.glob("*.png"):
  img = image.load_img(img)
  image_array = image.img_to_array(img)
  images.append(image_array)
  labels.append(1)

In [ ]:
for img in not_dog_path.glob("*.png"):
  img = image.load_img(img)
  image_array = image.img_to_array(img)
  images.append(image_array)
  labels.append(0)

In [ ]:
x_train = np.array(images)
y_train = np.array(labels)

In [ ]:
#normalise data between 0-1
x_train = vgg16.preprocess_input(x_train)

In [ ]:
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3) )

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
features = pretrained_nn.predict(x_train)

In [ ]:
# Save the array of extracted features to a file
joblib.dump(features, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

# Training with extracted features


In [ ]:
# Load data set
x_train_ex = joblib.load("x_train.dat")
y_train_ex = joblib.load("y_train.dat")

In [ ]:
x_train_ex.shape

In [ ]:
model=None
# Create a model and add layers

model = keras.models.Sequential()
model.add(Flatten(input_shape=x_train_ex.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 524,801
Trainable params: 524,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
model.fit(
    x_train_ex,
    y_train_ex,
    epochs=10,
    shuffle=True
)


Epoch 1/10
2/2 [==============================] - 0s 9ms/step - loss: 11.1232 - accuracy: 0.5172
Epoch 2/10
2/2 [==============================] - 0s 13ms/step - loss: 2.4288 - accuracy: 0.8793
Epoch 3/10
2/2 [==============================] - 0s 11ms/step - loss: 1.3304 - accuracy: 0.9655
Epoch 4/10
2/2 [==============================] - 0s 8ms/step - loss: 2.2996 - accuracy: 0.9655
Epoch 5/10
2/2 [==============================] - 0s 8ms/step - loss: 0.1622 - accuracy: 0.9828
Epoch 6/10
2/2 [==============================] - 0s 15ms/step - loss: 0.2874 - accuracy: 0.9828
Epoch 7/10
2/2 [==============================] - 0s 9ms/step - loss: 2.3388e-13 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 8ms/step - loss: 9.1097e-07 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 8ms/step - loss: 1.6470e-14 - accuracy: 1.0000


In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")

# Making Prediction

In [ ]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np
from keras.applications import vgg16

In [ ]:
z = Path("model_structure.json")

In [ ]:
model_structure_custom = z.read_text()
model_structure_custom

In [ ]:
custom_model = model_from_json(model_structure_custom)
custom_model

In [ ]:
custom_model.load_weights("model_weights.h5")

In [125]:
img = image.load_img("dog.png", target_size=(64, 64))
image_array = image.img_to_array(img)
images = np.expand_dims(image_array, axis=0)
images = vgg16.preprocess_input(images)

In [126]:
images.shape

(1, 64, 64, 3)

In [127]:
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))
features = feature_extraction_model.predict(images)

In [131]:
features.shape

(1, 2, 2, 512)

In [129]:
results = custom_model.predict(features)

In [130]:
single_result = results[0][0]
single_result

1.0